In [10]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import time
from typing import Dict, List, Any, Optional, Union

In [2]:
# API Configuration
API_KEY = "FzE3AJhBEiPRbnKHa6sQBPSEE9ByT6Zc"
BASE_URL = "https://api.sportsgameodds.com/v2"
HEADERS = {
    "X-Api-Key": API_KEY
}


In [3]:
def get_date(offset: int = 0) -> str:
    """
    Get a date with an offset from today in YYYY-MM-DD format.
    
    Args:
        offset: Number of days to offset from today (0=today, 1=tomorrow, -1=yesterday)
        
    Returns:
        Date string in YYYY-MM-DD format
    """
    date = datetime.now() + timedelta(days=offset)
    return date.strftime("%Y-%m-%d")

def get_tomorrow_date() -> str:
    """Get tomorrow's date in YYYY-MM-DD format."""
    return get_date(1)

def make_api_request(endpoint: str, params: Dict = None) -> Dict:
    """
    Make a request to the Sports Game Odds API.
    
    Args:
        endpoint: API endpoint to call
        params: Query parameters for the request
        
    Returns:
        JSON response from the API
    """
    url = f"{BASE_URL}/{endpoint}"
    print(f"Making request to: {url}")
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        response.raise_for_status()  # Raise an exception for 4XX/5XX responses
        
        # Add a small delay to avoid rate limiting
        time.sleep(1)
        
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API Request Error: {e}")
        return {"success": False, "error": str(e)}

# CHUNK 3: NBA EVENTS AND PLAYER PROPS
def get_nba_events(date: str = None, day_offset: int = None) -> pd.DataFrame:
    """
    Get NBA events for a specific date.
    
    Args:
        date: Date in YYYY-MM-DD format (defaults to today)
        day_offset: Day offset from today (-1=yesterday, 0=today, 1=tomorrow)
        
    Returns:
        DataFrame containing NBA events
    """
    # If day_offset is provided, use it to calculate the date
    if day_offset is not None:
        date = get_date(day_offset)
    # Otherwise use the provided date or default to today
    elif not date:
        date = get_today_date()
    
    params = {
        "leagueID": "NBA",
        "startsAfter": f"{date}T00:00:00Z",
        "startsBefore": f"{date}T23:59:59Z",
        "limit": 100
    }
    
    print(f"Fetching NBA events for {date} with params: {params}")
    response = make_api_request("events", params)
    
    if response.get("success"):
        events_data = response.get("data", [])
        return pd.DataFrame(events_data)
    else:
        print(f"Error fetching NBA events: {response.get('error')}")
        return pd.DataFrame()

In [4]:
def get_player_props(event_id: str) -> pd.DataFrame:
    """
    Get player props for a specific event.
    
    Args:
        event_id: Event ID to get props for
        
    Returns:
        DataFrame containing player props
    """
    params = {
        "eventID": event_id,
        "limit": 100
    }
    
    print(f"Fetching player props for event {event_id} with params: {params}")
    response = make_api_request("odds", params)
    
    if response.get("success"):
        odds_data = response.get("data", [])
        
        # Filter for player props only
        player_props = [odd for odd in odds_data if "PLAYER" in str(odd.get("oddID", ""))]
        return pd.DataFrame(player_props)
    else:
        print(f"Error fetching odds: {response.get('error')}")
        return pd.DataFrame()

In [7]:
def parse_underdog_html(html_file):
    """Parse Underdog Fantasy HTML to extract player props."""
    with open(html_file, 'r') as f:
        html_content = f.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    props = []
    
    # This is a placeholder - you'll need to inspect Underdog's HTML structure
    # and adjust these selectors accordingly
    prop_cards = soup.select('.player-card') # Adjust selector based on actual HTML
    
    for card in prop_cards:
        try:
            # Extract player name
            player_name = card.select_one('.player-name').text.strip()
            
            # Extract game info (optional, for context)
            game_info = card.select_one('.game-info').text.strip()
            
            # Extract prop details
            prop_elements = card.select('.prop-row')
            
            for prop_elem in prop_elements:
                prop_type = prop_elem.select_one('.stat-type').text.strip()
                line = float(prop_elem.select_one('.line-value').text.strip())
                
                # Extract over/under options
                over_elem = prop_elem.select_one('.higher')
                under_elem = prop_elem.select_one('.lower')
                
                # Add over prop
                props.append({
                    'player': player_name,
                    'game': game_info,
                    'prop_type': prop_type,
                    'line': line,
                    'direction': 'over',
                    'source': 'Underdog'
                })
                
                # Add under prop
                props.append({
                    'player': player_name,
                    'game': game_info,
                    'prop_type': prop_type,
                    'line': line,
                    'direction': 'under',
                    'source': 'Underdog'
                })
                
        except Exception as e:
            print(f"Error parsing prop card: {e}")
            continue
    
    return pd.DataFrame(props)

In [11]:
UNDERDOG_HTML = "underdog_fantasy.html"
    
# 1. Parse Underdog data
print("Parsing Underdog Fantasy data...")
underdog_props = parse_underdog_html(UNDERDOG_HTML)
print(f"Found {len(underdog_props)} props on Underdog")

Parsing Underdog Fantasy data...
Found 0 props on Underdog
